In [1]:
import torch 
import numpy as np
from typing import Sequence

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from evaluate import evaluate
from data.kernel import *
from inspect import getmembers, isfunction


%reload_ext autoreload
%autoreload 2

In [2]:
from itertools import combinations

In [17]:
x = torch.rand(2, 3)
bound = torch.tensor([[0, 5], [6, 10], [11, 15]]).T
print(bound)
print(x)
x_scale = bound[0] + x.clone() * (
                bound[1] - bound[0]
            )  # NOTE scale X to test range
print(x_scale)

tensor([[ 0,  6, 11],
        [ 5, 10, 15]])
tensor([[0.2833, 0.0578, 0.8244],
        [0.5581, 0.7745, 0.6147]])
tensor([[ 1.4166,  6.2312, 14.2975],
        [ 2.7904,  9.0980, 13.4589]])


In [12]:
num_points = 10
n_comp = 3
y = torch.randn(num_points, 1)
all_combinations = np.array(list(combinations(range(y.shape[0]), 2)))
comp = all_combinations[
            np.random.choice(range(len(all_combinations)), n_comp, replace=False)
        ]
print(comp)
comp_y = y[comp].squeeze(-1)
# squeeze(-1) # (num_points, num_comp, 2)
print(comp_y.shape)
flip_indices = (
            (comp_y[..., 0] > comp_y[..., 1]).squeeze(-1).cpu().numpy()
        )
print(comp_y)
print(flip_indices)
comp[flip_indices, :] = np.flip(comp[flip_indices, :], 1)
print(comp)

[[2 4]
 [4 7]
 [0 9]]
torch.Size([3, 2])
tensor([[ 1.0223, -2.2910],
        [-2.2910,  0.3795],
        [ 0.2926, -0.2054]])
[ True False  True]
[[4 2]
 [4 7]
 [9 0]]


In [2]:
from utils.log import TensorAverager

In [3]:
ravg = TensorAverager()

In [7]:
D = 1
bx = list()
ravg.clear()
for t in range(7): 
    x = torch.randint(0,10,(1,))
    print(x)
    bx.append(x)
    ravg.update("x", x)


tensor([3])
tensor([2])
tensor([6])
tensor([1])
tensor([2])
tensor([9])
tensor([9])


In [8]:
bx = torch.stack(bx, dim=0).float().mean(dim=0)
print(bx)

tensor([4.5714])


In [14]:
tmp = [torch.randn(1) for _ in range(6)]
tmp = torch.cat(tmp, dim=0)
print(tmp.shape)

torch.Size([6])


In [9]:
ax = ravg.get("x")
print(ax)

tensor([4.5714])


In [ ]:

print(globals()["rbf"])

In [ ]:
functions_list = getmembers(kernel, isfunction)
print(functions_list)

In [ ]:
from data.utils import *
from PABBO.data.sampler import * 


data = torch.load("datasets/GP2D_initial_pairs.pt")["0"]

fig = plt.figure(figsize=(18, 5))
for b in range(4):
    sampler = SimpleGPSampler(
                    kernel_function=globals()[
                        data["sampler_kwargs"][b]["kernel_function"]
                    ],
                    mean=data["sampler_kwargs"][b]["mean"],
                    jitter=data["sampler_kwargs"][b]["jitter"],
                )
    utility = OptimizationFunction(
                    sampler=sampler, **data["function_kwargs"][b]
    )


    pred_train_y = utility(utility.train_X)

    ax = fig.add_subplot(1, 4, b+1, projection='3d')
    ax.scatter(utility.train_X[:, 0], utility.train_X[:, 1], pred_train_y, alpha=0.1)
    opt_y = utility(utility.Xopt)
    ax.scatter(utility.Xopt[:, 0], utility.Xopt[:, 1], opt_y, c="orange")


We provide a minimal implementation of the source code to reproduce our ablation experiments on meta-training datasets. While we have tried our best to make the code as clear and instructive as possible within the limited time, we apologize for any potential oversights or errors.

**Dependencies**: `pip install -r requirements`

**Experiments**: To evaluate a specific PABBO model, you can use the `evaluate` function from `evaluate.py` as follows:

1. Command Line Execution. 
    
    Run the command:
    ```bash 
    python evaluate.py --model_name PABBO_RBF
    ```
    Here, `model_name` can be either "PABBO_RBF" or "PABBO_Mixture". For convenience, a batch script (`run.sh`) is also provided.

2. Notebook Execution. 
    
    Alternatively, you can execute this notebook. Note that this approach may take quite an amount of time.

Results will be saved in the directory `evaluation/results/model_name` and will include the following three files:

- `SIMPLE_REGRET.pt`
- `STEP_SIMPLE_REGRET.pt`
- `CUMULATIVE_REGRET.pt`

Each file contains a `torch.tensor` of shape `(num_seed, T)`, representing the regrets collected along the trajectory, averaged across tasks, and evaluated under different random seeds.


In [ ]:
override = False
if override: 
    SIMPLE_REGRET_RBF, SIMPLE_STEP_REGRET_RBF, CUMULATIVE_REGRET_RBF = evaluate(model_name="PABBO_RBF")
    SIMPLE_REGRET_Mixture, SIMPLE_STEP_REGRET_Mixture, CUMULATIVE_REGRET_Mixture = evaluate(model_name="PABBO_Mixture")
else: 
    SIMPLE_REGRET_RBF = torch.load("evaluation/results/PABBO_RBF/SIMPLE_REGRET.pt")
    CUMULATIVE_REGRET_RBF = torch.load("evaluation/results/PABBO_RBF/CUMULATIVE_REGRET.pt")
    SIMPLE_REGRET_Mixture = torch.load("evaluation/results/PABBO_Mixture/SIMPLE_REGRET.pt")
    CUMULATIVE_REGRET_Mixture = torch.load("evaluation/results/PABBO_Mixture/CUMULATIVE_REGRET.pt")

Plot the results:

In [ ]:
def confidence_interval(value: np.ndarray):
    return 1.96 * value.std(axis=0) / np.sqrt(value.shape[0])

def plot_trajectory_information(
    trajectories: Sequence[np.ndarray], 
    model_names: Sequence, 
    ax, 
):
    """plot optmimzation collected along trajectories.
    Args: 
        trajectories, list: a list of np.ndarray of shape (B, T).
        model_names, list: a list of model names.
    """
    M = len(trajectories)
    colors = cm.rainbow(np.linspace(0, 1, M))

    for i, (model_tra, c) in enumerate(zip(trajectories, colors)):
        x = np.arange(1, model_tra.shape[-1] + 1)
        mean = model_tra.mean(axis=0).flatten()
        ci = confidence_interval(model_tra)
        ax.plot(x, mean, "o-", color=c, label=f"{model_names[i]}")
        ax.fill_between(x, mean + ci, mean - ci, alpha=0.3, color=c)
        
    return ax


In [ ]:
SIMPLE_REGRET_TRA = [SIMPLE_REGRET_RBF, SIMPLE_REGRET_Mixture]
CUMULATIVE_REGRET_TRA = [CUMULATIVE_REGRET_RBF, CUMULATIVE_REGRET_Mixture]
model_names = ["PABBO RBF", "PABBO Mixture"]

fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(1, 2, 1)
ax = plot_trajectory_information(SIMPLE_REGRET_TRA, model_names, ax)
ax.set_ylabel("Simple regret")
ax.set_xlabel("Step")
ax.set_yscale("log")
ax = fig.add_subplot(1, 2, 2)
ax = plot_trajectory_information(CUMULATIVE_REGRET_TRA, model_names, ax)
ax.set_yscale("log")
ax.set_ylabel("Cumulative regret")
ax.set_xlabel("Step")

# fig.suptitle("PABBO RBF v.s. PABBO Mixture")
handles, labels = fig.axes[0].get_legend_handles_labels()
fig.legend(handles, labels, ncol=len(labels), loc="lower center", bbox_to_anchor=(0.5, -0.05))
plt.tight_layout()
plt.show()